In [1]:
import numpy as np
import pandas as pd
from tortreinador.utils.tools import check_outlier
from tortreinador.utils.plot import plot_line_2
from tortreinador.utils.preprocessing import load_data
from tortreinador.train import TorchTrainer
from tortreinador.models.MDN import Mixture, NLLLoss
from Rock.Model.EnsembleMDN import EnsembleMDN
from tortreinador.utils.View import init_weights, split_weights
from tortreinador.utils.plot import calculate_GMM
import torch
import torch.nn as nn
from tortreinador.utils.metrics import r2_score
import matplotlib
from matplotlib import pyplot as plt
import proplot as pplt
import joblib

In [2]:
df_chunk_0 = pd.read_parquet("D:\\Resource\\rockyExoplanetV3\\data_chunk_0.parquet")
df_chunk_1 = pd.read_parquet("D:\\Resource\\rockyExoplanetV3\\data_chunk_1.parquet")

df_all = pd.concat([df_chunk_0, df_chunk_1])

In [3]:
input_parameters = [
    'Mass',
    'Radius',
    'FeMg',
    'SiMg',
    'Mass_Noise',
    'Radius_Noise',
    'FeMg_Noise',
    'SiMg_Noise'
]

output_parameters = [
    'WRF',
    'MRF',
    'CRF',
    'WMF',
    'CMF',
    'CPS',
    'CTP',
    'k2'
]

In [4]:
offset_rate = np.array([0.04, 0.02, 0.12, 0.14])

In [5]:
input_df_4 = df_all[input_parameters[:4]]

# calculate the sigma
sigma = input_df_4.mul(offset_rate)

In [6]:
def generate_noise(x):
    return float(np.random.normal(0, x, size=1))

In [7]:
# Generate Noise
noise = pd.DataFrame()
for i in input_parameters[:4]:
    tmp = sigma[i].apply(generate_noise)
    noise = pd.concat([noise, tmp], axis=1)

# Add Noise
noise_added_data = input_df_4 + noise
noise_added_data.columns = [
    'Mass_Noise',
    'Radius_Noise',
    'FeMg_Noise',
    'SiMg_Noise',
]

df_merged = pd.concat([df_all, noise_added_data], axis=1)

In [8]:
t_loader, v_loader, test_x, test_y, s_x, s_y = load_data(data=df_merged, input_parameters=input_parameters,
                                                         output_parameters=output_parameters,
                                                         if_normal=True, if_shuffle=True, batch_size=256)

In [9]:
# Model
model = EnsembleMDN(input_size=int(len(input_parameters) / 2), output_size=len(output_parameters), num_hidden=256, num_gaussian=20, kernel_size=2)
init_weights(model)
criterion = NLLLoss()
optimizer = torch.optim.Adam(split_weights(model), lr=0.0001, weight_decay=0.0001)
mixture = Mixture()

In [10]:
class Trainer(TorchTrainer):
    def calculate(self, x, y, mode='t'):
        x_o, x_n = x.chunk(2, dim=1)
        
        pi, mu, sig = model(x_o, x_n)
        
        loss = self.criterion(pi, mu, sig, y)
        pdf = mixture(pi, mu, sig)
        y_pred = pdf.sample()
        
        metric_per = r2_score(y, y_pred)
        return self._standard_return(loss=loss, metric_per=metric_per, mode=mode, y=y, y_pred=y_pred)
        

In [11]:
trainer = Trainer(is_gpu=True, epoch=200, optimizer=optimizer, model=model, criterion=criterion)

Epoch:200, is GPU: True


In [12]:
config = {
    'b_m': 0.8,
    'm_p': 'D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\',
    'w_e': 5,
    # 'l_m': {
    #     's_l': [18, 48, 84, 100, 113, 124, 133],
    #     'gamma': 0.7
    # }
}

In [13]:
result = trainer.fit(t_loader, v_loader, **config)

module.mdn_original.root_layer.0.weight : torch.Size([256, 4])
module.mdn_original.root_layer.0.bias : torch.Size([256])
module.mdn_original.root_layer.2.weight : torch.Size([256, 256])
module.mdn_original.root_layer.2.bias : torch.Size([256])
module.mdn_original.root_layer.4.weight : torch.Size([256, 256])
module.mdn_original.root_layer.4.bias : torch.Size([256])
module.mdn_original.pi.0.weight : torch.Size([256, 256])
module.mdn_original.pi.0.bias : torch.Size([256])
module.mdn_original.pi.2.weight : torch.Size([20, 256])
module.mdn_original.pi.2.bias : torch.Size([20])
module.mdn_original.mu.0.weight : torch.Size([256, 256])
module.mdn_original.mu.0.bias : torch.Size([256])
module.mdn_original.mu.2.weight : torch.Size([160, 256])
module.mdn_original.mu.2.bias : torch.Size([160])
module.mdn_original.sigma.0.weight : torch.Size([256, 256])
module.mdn_original.sigma.0.bias : torch.Size([256])
module.mdn_original.sigma.2.weight : torch.Size([160, 256])
module.mdn_original.sigma.2.bias :

  0%|          | 0/11025 [00:00<?, ?batch/s]E:\anaconda\envs\deeplearning\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Epoch 1 Training:   0%|          | 0/11025 [00:05<?, ?batch/s]


RuntimeError: expected scalar type Double but found Float

In [ ]:
result_pd = pd.DataFrame()
result_pd['epoch'] = range(200)
result_pd['train_loss_avg'] = result[0].val.detach().cpu().numpy()
result_pd['validation_loss_avg'] = result[1].val.detach().cpu().numpy()

plot_line_2(y_1='train_loss_avg', y_2='validation_loss_avg', df=result_pd.iloc[3:, :], output_path=".\\imgs\\ROCKYEXO_ENABLEMDN20240630_TrainValLoss_2.png")

In [ ]:
check_outlier(result[1].val.detach().cpu().numpy(), 1, 15, 3.0)

In [ ]:
np.save('D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\test_x.npy', test_x)
np.save('D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\test_y.npy', test_y)
joblib.dump(s_x, "D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\MDN_v3_Xscaler_20240630.save")
joblib.dump(s_y, "D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\MDN_v3_yscaler_20240630.save")

In [ ]:
t_x = np.load("D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\test_x.npy")
t_y = np.load("D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\test_y.npy")
m_y = joblib.load("D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\testData\\MDN_v3_yscaler_20240630.save")

In [ ]:
model = EnsembleMDN(int(len(input_parameters) / 2), len(output_parameters), 20, 512, kernel_size=3)
init_weights(model)
model = nn.DataParallel(model)
model.load_state_dict(torch.load("D:\\Resource\\MDN\\rockyExoplanetV3\\NoiseADD\\best_model.pth"))

In [ ]:
rand_idx = torch.randint(0, t_x.shape[0], size=(20000, ))

In [ ]:
sampled_test_x = t_x[rand_idx]
sampled_test_y = t_y[rand_idx]

In [ ]:
t_x_o, t_x_n = np.array_split(sampled_test_x, 2, axis=1)

In [ ]:
model.eval()
pred = model(torch.from_numpy(t_x_o), torch.from_numpy(t_x_n))

In [ ]:
mix = Mixture()
criterion = NLLLoss()
mse = nn.MSELoss()
sample = mix(pred[0], pred[1], pred[2]).sample()
print("NLLLoss: {}, MSE: {}, R2: {}".format(criterion(pred[0], pred[1], pred[2], torch.from_numpy(sampled_test_y).to('cuda')),
                                            mse(torch.from_numpy(sampled_test_y).to('cuda'), sample),
                                            r2_score(sample, torch.from_numpy(sampled_test_y).to('cuda'))))

In [ ]:
GMM_PDF_scaled = calculate_GMM(torch.exp(pred[0]).detach().cpu().numpy(), pred[1].detach().cpu().numpy(), pred[2].detach().cpu().numpy())

In [ ]:
sampled_test_y_inverse = s_y.inverse_transform(sampled_test_y)

In [ ]:
cmap = plt.cm.hot_r
norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
fig, axs = pplt.subplots(
#     figsize=(4,4),
    nrows=2, ncols=4,
    share=False, 
    figsize=(12, 8)
#     tight=True,
)

output_parameters = [
    'WRF',
    'MRF',
    'CRF',
    'WMF',
    'CMF', 
    'CPS',
    'CTP',
    'k2'
]


xlabels = [
    "Actual WRF","Actual MRF", "Actual CRF", 
    "Actual WMF","Actual CMF", "Actual CMB pressure (GPa)", "Actual CMB temperature (K)",
    "Actual k2", 
]
ylabels = [
    "Predicted WRF","Predicted MRF", "Predicted CRF", 
    "Predicted WMF","Predicted CMF", "Predicted CMB pressure (GPa)", "Predicted CMB temperature (K)",
    "Predicted k2", 
]

xlocators = [
    0.04, 0.2, 0.2, 0.02, 0.2, 400, 1000, 0.2
]
xminorlocators = [
    0.004, 0.02, 0.02, 0.02, 40, 100, 0.04, 0.004
]

OUTPUT_DIMS = len(output_parameters)

for o in range(OUTPUT_DIMS):
    y_max = max(sampled_test_y_inverse[:, o])
    y_min = min(sampled_test_y_inverse[:, o])
    for i in range(0, GMM_PDF_scaled.shape[-1], OUTPUT_DIMS):
        tx, ty = [sampled_test_y_inverse[int(i / OUTPUT_DIMS), o], y_min]
        axs[o].imshow(
                GMM_PDF_scaled[:, o + i].reshape(-1, 1),
                cmap=cmap,
                norm=norm,
                origin='lower',
                extent=(tx, tx + 0.0001, ty, y_max)
            )

    axs[o].plot([y_min, y_max], [y_min, y_max], c='cornflowerblue', ls='--')
    axs[o].format(
        xlim=(y_min, y_max), ylim=(y_min, y_max),
        xlabel=xlabels[o], ylabel=ylabels[o],
        xlocator=xlocators[o], xminorlocator=xminorlocators[o],
        # ylocator=xlocators[o], yminorlocator=xminorlocators[o]
    )

In [ ]:
# Calculate the offset of the probability density heatmap
col_dis = []
for o in range(len(output_parameters)):
    dis = 0
    for i in range(0, GMM_PDF_scaled.shape[-1], len(output_parameters)):
        test_y_current = sampled_test_y_inverse[int(i / len(output_parameters)), o]
        GMM_cal = GMM_PDF_scaled[:, o + i].reshape(-1, 1)
        offset = np.sqrt((GMM_cal - test_y_current) ** 2)
        dis += np.mean(offset)
    col_dis.append(dis / len(sampled_test_y_inverse))